## **YOUTUBE API COMMENT SCRAPER**


# Gathers Data

In [1]:
#boilerplate code, run first in order to get the code running
#Special thanks to NAME for help, refer to email correspondence on 3.3.2021 jlacuesta@mail.sfsu.edu

!pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from ipywidgets import FileUpload
import urllib.parse as p
import re
import os
import pickle
from google.colab import files

#Input for Sentiment Analysis
#View more about vaderSentiment here ------> https://github.com/cjhutto/vaderSentiment/blob/master/README.rst#installation
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

#Insert your API keys here

SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl'] #this conforms the API key
uploaded = files.upload() #this SECOND uploads it, needs to be named 'client_secret.json'

     |████████████████████████████████| 6.5MB 12.4MB/s 
ERROR: earthengine-api 0.1.258 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 2.0.2 which is incompatible.
  Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Found existing installation: google-auth-oauthlib 0.4.3
    Uninstalling google-auth-oauthlib-0.4.3:
      Successfully uninstalled google-auth-oauthlib-0.4.3
     |████████████████████████████████| 133kB 9.8MB/s 


Saving client_secret.json to client_secret.json


In [10]:
#This scrapes the comments from videos
#Reference ----> https://stackoverflow.com/questions/34606055/how-to-get-comments-from-videos-using-youtube-api-v3-and-python

import os
import pickle
import google.oauth2.credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import io

CLIENT_SECRETS_FILE = "client_secret.json" 
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def get_video_comments(service, **kwargs):
    comments = []
    results = service.commentThreads().list(**kwargs).execute()
    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)
        # Check if another page exists
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = service.commentThreads().list(**kwargs).execute()
        else:
            break

    return comments

if __name__ == '__main__':
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.
    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()
    videoId = input('Enter Video id : ') # video id here (the video id of https://www.youtube.com/watch?v=vedLpKXzZqE -> is vedLpKXzZqE)
    comments = get_video_comments(service, part='snippet', videoId=videoId, textFormat='plainText', order = 'time')
    analyzer = SentimentIntensityAnalyzer()
    scores = []
    print(videoId)

    data1 = pd.DataFrame(comments, columns= ['Comment'])
    print(data1)

data1.to_csv('drive/My Drive/ExportTest/DFTest3.csv') #REMEMBER to change the name based on the codes

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter Video id : WI0RTUeHToo
WI0RTUeHToo
                                               Comment
0                  1:00 Ren (to Stimpy): YOU ANTA BAKA
1                Wait, they're speaking english tho...
2             Que asco de doblaje era mejor el primero
3    You can add the 1995 dinyt dubbing so, in my o...
4         アスカなら一人で二号機に乗り込みそうだけどな。\n\nなんでシンジを連れてったんだっけ？
..                                                 ...
108                               Tem até em português
109  Muy ofendido que no hayan esperado hasta mañan...
110  GKIDS, if you can hear me, please have both th...
111                                      The dub sucks
112                         Yes Netflix finally did it

[113 rows x 1 columns]
